In [5]:
# load carbon traces into a pandas dataframe
import pandas as pd
import numpy as np

In [6]:
# AWS regions
names = [
    "us-east-1",      # US East (N. Virginia)
    "us-west-1",      # US West (N. California)
    "us-west-2",      # US West (Oregon)
    "af-south-1",     # Africa (Cape Town)
    "ap-south-2",     # Asia Pacific (Hyderabad)
    "ap-northeast-2", # Asia Pacific (Seoul)
    "ap-southeast-2", # Asia Pacific (Sydney)
    "ca-central-1",   # Canada (Central)
    "eu-central-1",   # Europe (Frankfurt)
    "eu-west-2",      # Europe (London)
    "eu-west-3",      # Europe (Paris)
    "eu-north-1",     # Europe (Stockholm)
    "sa-east-1",       # South America (São Paulo)
    "il-central-1"    # Israel (Tel Aviv)
]

marginal_names = [
    "us-east-1",      # US East (N. Virginia)
    "us-west-1",      # US West (N. California)
    "us-west-2",      # US West (Oregon)
    "ap-southeast-2", # Asia Pacific (Sydney)
    "ca-central-1",   # Canada (Central)
    "eu-central-1",   # Europe (Frankfurt)
    "eu-west-2",      # Europe (London)
    "eu-west-3",      # Europe (Paris)
    "eu-north-1",     # Europe (Stockholm)
]

In [7]:
# define a dict of dataframes
dfs = {}
for name in names:
    df = pd.read_csv(f"carbon-data/{name}.csv", parse_dates=["datetime"])
    # keep only the columns we need
    df = df[["datetime", "carbon_intensity_avg"]]
    dfs[name] = df
    # print(len(df))

# print the first few rows of the first dataframe
print(dfs["us-east-1"].head())

                   datetime  carbon_intensity_avg
0 2020-01-01 00:00:00+00:00                382.14
1 2020-01-01 01:00:00+00:00                377.11
2 2020-01-01 02:00:00+00:00                375.74
3 2020-01-01 03:00:00+00:00                373.20
4 2020-01-01 04:00:00+00:00                360.48


In [41]:
# define a dict of dataframes
dfs = {}
for name in marginal_names:
    df = pd.read_csv(f"marginal-data/{name}.csv", parse_dates=["datetime"])
    # keep only the columns we need
    df = df[["datetime", "marginal_carbon_avg", "marginal_forecast_avg"]]
    # anything below 12 is a data error, truncate it to 12
    df["marginal_carbon_avg"] = np.maximum(df["marginal_carbon_avg"], 1)
    df["marginal_forecast_avg"] = np.maximum(df["marginal_forecast_avg"], 1)
    dfs[name] = df
    # print(len(df))

# print the first few rows of the first dataframe
print(dfs["us-east-1"].head())

                   datetime  marginal_carbon_avg  marginal_forecast_avg
0 2022-01-01 00:00:00+00:00          1215.583333            1215.527228
1 2022-01-01 01:00:00+00:00          1201.333333            1215.846383
2 2022-01-01 02:00:00+00:00          1201.250000            1217.291609
3 2022-01-01 03:00:00+00:00          1177.916667            1217.602517
4 2022-01-01 04:00:00+00:00          1177.333333            1218.497541


In [42]:
region = "ap-southeast-2"
dfs[region]["marginal_carbon_avg"].min()

1.0

In [43]:
dfs[region]["marginal_carbon_avg"].max()

1949.5

In [70]:
# print the global average carbon intensity
global_avg = np.mean([df["carbon_intensity_avg"].mean() for df in dfs.values()])
print(f"Global average carbon intensity: {global_avg} gCO2/kWh")

Global average carbon intensity: 362.0820754894641 gCO2/kWh


In [1]:
# load the metric space
import metric
m = metric.MetricSpace()

# get the "column names" of the vectors for the tree embedding in the metric space
# name_vector = m.name_vector

In [2]:
# get simplex distances from the metric space
simplex_names, c_simplex, simplex_distances = m.generate_simplex_distances()

# get weight vector from the metric space
weight_vector = m.weight_vector

print(simplex_distances)

[[  0.           1.         221.04270463 221.04270463 265.87320574
  265.87320574 258.25058548 258.25058548 722.05950992 722.05950992
  772.2161215  772.2161215  833.40118343 833.40118343 423.50058893
  423.50058893 375.51670644 375.51670644 299.64739884 299.64739884
  328.75874126 328.75874126 461.9706228  461.9706228  483.33471074
  483.33471074 686.         686.        ]
 [  1.           0.         221.04270463 221.04270463 265.87320574
  265.87320574 258.25058548 258.25058548 722.05950992 722.05950992
  772.2161215  772.2161215  833.40118343 833.40118343 423.50058893
  423.50058893 375.51670644 375.51670644 299.64739884 299.64739884
  328.75874126 328.75874126 461.9706228  461.9706228  483.33471074
  483.33471074 686.         686.        ]
 [221.04270463 221.04270463   0.           1.         127.71512309
  127.71512309 333.99182243 333.99182243 399.68632075 399.68632075
  548.81081081 548.81081081 597.69266055 597.69266055 295.36737089
  295.36737089 666.83783784 666.83783784 639.

In [73]:
def weighted_l1_norm(vector1, vector2, phi, weights, cvxpy=False):
    if cvxpy:
        phi1 = phi @ vector1
        phi2 = phi @ vector2
        weighted_diff = cp.multiply(cp.abs(phi1 - phi2), weights)
        weighted_sum = cp.sum(weighted_diff)
    else:
        phi1 = phi @ vector1
        phi2 = phi @ vector2
        weighted_diff = np.abs(phi1 - phi2) * weights
        weighted_sum = np.sum(weighted_diff)

    return weighted_sum

In [85]:
# choose two regions to measure distance (before)
region1 = "us-west-1"
region2 = "ap-northeast-2"

# get the distance between the two regions
distance = m.distance(region1, region2)

weights = m.get_weight_vector()

c_vector, name_vector = m.get_unit_c_vector()

print(f"Distance between {region1} and {region2}: {distance}")

# get the distance between the two regions (vector)
east_vector, east_simplex = m.get_start_state(region1)
west_vector, west_simplex = m.get_start_state(region2)
phi = m.phi(names, name_vector, simplex_names)

print(phi.shape)
print(len(east_vector))
print(len(west_vector))

vector_distance = weighted_l1_norm(east_simplex, west_simplex, phi, weights)
print(f"Vector distance between {region1} and {region2}: {vector_distance}")

Distance between us-west-1 and ap-northeast-2: 548.8108108108108
(35, 28)
35
35
Vector distance between us-west-1 and ap-northeast-2: 1297.3890153320299


In [86]:
# let's try swapping us-west-1 with us-west-2
# create a new simplex distance matrix
new_simplex_distances = simplex_distances.copy()
swap1index = simplex_names.index("us-west-1 OFF")
swap2index = simplex_names.index("eu-west-3 OFF")
new_simplex_distances[:, swap2index] = simplex_distances[:, swap1index]
new_simplex_distances[:, swap1index] = simplex_distances[:, swap2index]
new_simplex_distances[swap2index, :] = simplex_distances[swap1index, :]
new_simplex_distances[swap1index, :] = simplex_distances[swap2index, :]

# print the distance between us-west-1 and ap-northeast-2
distance = new_simplex_distances[simplex_names.index("eu-west-3 OFF"), simplex_names.index("ap-northeast-2 OFF")]

print(f"Distance between eu-west-3 and ap-northeast-2: {distance}")

Distance between eu-west-3 and ap-northeast-2: 548.8108108108108


In [87]:
# now swap the corresponding indices in the weight vector
new_name_vector = name_vector.copy()
for i, name in enumerate(name_vector):
    new_name = name.replace("us-west-1", "INTERMEDIATE").replace("eu-west-3", "us-west-1").replace("INTERMEDIATE", "eu-west-3")
    new_name_vector[i] = new_name

# get the distance between the two regions (vector)
phi = m.phi(names, new_name_vector, simplex_names)
east_vector, east_simplex = m.get_start_state("eu-west-3")
west_vector, west_simplex = m.get_start_state("ap-northeast-2")

vector_distance = weighted_l1_norm(east_simplex, west_simplex, phi, weights)
print(f"Vector distance between eu-west-3 and ap-northeast-2: {vector_distance}")


Vector distance between eu-west-3 and ap-northeast-2: 1297.3890153320299


In [66]:
print(weights)

print(name_vector)

[  0.         555.25949903 555.25949903 555.25949903 277.62974951
 277.62974951 277.62974951 277.62974951 277.62974951 277.62974951
 277.62974951 277.62974951 277.62974951 277.62974951 138.81487476
 138.81487476   1.           1.           1.         138.81487476
   1.           1.           1.         138.81487476 138.81487476
 138.81487476   1.           1.          69.40743738  69.40743738
   1.           1.          69.40743738  69.40743738   1.
   1.           1.           1.        ]
['root', "['ap-south-2', 'eu-north-1', 'sa-east-1', 'eu-west-2', 'eu-central-1', 'ca-central-1', 'af-south-1', 'ap-southeast-2', 'ap-northeast-2', 'us-west-1', 'eu-west-3', 'il-central-1', 'us-east-1', 'us-west-2']", "['ap-south-2', 'eu-north-1', 'sa-east-1', 'eu-west-2', 'eu-central-1', 'ca-central-1', 'af-south-1', 'ap-southeast-2', 'ap-northeast-2', 'us-west-1', 'eu-west-3', 'il-central-1', 'us-east-1', 'us-west-2']", "['ap-south-2', 'eu-north-1', 'sa-east-1', 'eu-west-2', 'eu-central-1', 'ca-cent

In [3]:
# get random distances from metric
rand_distances, _, rand_phi = m.shuffled_distances()

In [5]:
print(rand_distances.shape)

(28, 28)


In [6]:
print(simplex_distances.shape)

(28, 28)
